## Veri Setini Tanıyalım

* Veri setimizde bir bölgedeki evlerin özellikleri ve fiyat bilgileri yer almaktadır.Bu özellikler ise konum bilgisi,toplam oda sayısı,okyanusa yakınlık gibi bilgilerdir.
* Bizim burada amacımız ev fiyatlarını etkileyen sütunları kullanarak ev fiyatlarını tahmin edebilecek modeller geliştirmektir.
* Modelleri kurmadan önce verimizi ve gerekli kütüphaneleri içeri aktaralım.
* Veri seti hakkında bazı sorgular gerçekleştirerek veri ön işleme aşamalarını uygulayalım.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from catboost import CatBoostRegressor
from sklearn.linear_model import LinearRegression,RidgeCV,Ridge,Lasso,LassoCV,ElasticNet,ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error,r2_score
import xgboost
import lightgbm
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import plotly_express as px
import plotly

In [ ]:
df=pd.read_csv("../input/california-housing-prices/housing.csv")
df.head()

In [ ]:
df.info()

## Eksik Veri Tespit Etme ve Değer Atama
* Veri setlerinde eksik veri(missing values) yani boş DataFrame(nan) hücreleri bulunabilir.
* Bir veri setinde bulunan eksik verileri göz ardı ederek çalışmak doğru bir seçenek değildir.
* Eksik veriler silinebilir, aritmetik ortalama ,mod ya da medyan değerleri ile doldurulabilir.
* `total_bedrooms` değişkeninde 207 tane eksik veri mevcuutur.
* Bu çalışmada kategorik değişken olan `ocean_proximity` göre total_bedrooms değişkenlerini gruplandırarak eksik değerlere bu grupların ortlama değerlerini atayacağız.

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby("ocean_proximity")["total_bedrooms"].mean()

In [ ]:
df['total_bedrooms'] = df['total_bedrooms'].fillna(df.groupby('ocean_proximity')['total_bedrooms'].transform('mean'))

In [ ]:
df.isnull().sum()

## Coğrafi Veriyi Görselleştirme
* Verimizde yer alan konum bilgisinden yararlanarak verilerimizi harita üzerinde gösterelim.

In [ ]:
access_token = 'pk.eyJ1IjoiYWJkdWxrZXJpbW5lc2UiLCJhIjoiY2s5aThsZWlnMDExcjNkcWFmaWUxcmh3YyJ9.s-4VLvmoPQFPXdu9Mcd6pA'
px.set_mapbox_access_token(access_token)

In [ ]:
fig = px.scatter_mapbox(
    df, lat="latitude", lon="longitude",
    size="median_house_value", size_max=15,
    color="ocean_proximity", color_continuous_scale=px.colors.cyclical.IceFire,
    hover_name="ocean_proximity",           
    zoom=3,
    width=600, height=600
)
fig.layout.coloraxis.showscale = False
fig.update_layout(mapbox_style="open-street-map")
fig.show()

## Korelasyon Matrisi
* Korelasyon matrisinde değerler, -1 ile +1 arasındadır. -1' e yakın değerler negatif yöndeki ilişkiyi, +1'e yakın değerler pozitif yöndeki ilişkiyi temsil eder.
* Pozitif korelasyona sahip iki değişkenin değerleri birlikte artar veya birlikte azalır.
* Negatif korelasyona sahip iki değişkenlerden birinin değeri artarken diğerininki azalır.
* Değerin 0'a yakın olması durumu ise bu iki değişken arasında bir bağlantı olmadığını gösterir.

In [ ]:
corr = df.iloc[:,2:,].corr()
plt.figure(figsize=(8,8))
sns.heatmap(corr, annot = True)
plt.show()

## Catplot
* Aşağıdaki grafik ise okyanusa yakınlık derecelerine göre evlerin fiyatlarının hangi fiyat aralığında yoğunlaştığını anlamamıza olanak sağlar.
* Örneğin ada da bulunan evlerin ortlama fiyatları 200000 ile 500000 arasında yer almaktadır.


In [ ]:
sns.catplot(x="ocean_proximity",kind='violin', y="median_house_value",data=df);

## Aykırı Verileri Tespit Etme ve Değer Atama
* Aykırı veriler; bir veri setindeki gözlemlerin geri kalanından büyük ölçüde farklı olan gözlemlerdir.
* Aykırı gözlemleri görselleştirmek için bir çok teknik var biz burada en yaygın olan kutu grafiği ( Box Plot ) kullanacağız.
* Aykırı gözlemleri belirleyebilmemiz için box plotta gördüğünüz gibi sınır değerlerini tanımlamamız gerekiyor. Bunun için IQR tekniğini kullanıyoruz.
* Q1, değişkenin ilk çeyrelik kısmı ( 0.25 )
* Q3, değişkenin üçüncü çeyrekliği oluyor ( 0. 75 )
* Q3-Q1 ile IQR değerini hesaplıyoruz.
* Q1-Q3-IQR tamam şimdi sırada alt sınır ve üst sınır değerlerinde.
* Alt sınır : Q1–1.5*IQR
* Üst sınır : Q3 + 1.5*IQR
* Alt sınırın altındaki ve üst sınırın üstündeki değerler aykırı değer olarak tanımlanacak


In [ ]:
plt.figure(figsize=(12,12))
plt.subplot(3,2,1)
sns.boxplot(x='total_rooms',data=df)
plt.subplot(3,2,2)
sns.boxplot(x='total_bedrooms',data=df)
plt.subplot(3,2,3)
sns.boxplot(x='population',data=df)
plt.subplot(3,2,4)
sns.boxplot(x='households',data=df)
plt.subplot(3,2,5)
sns.boxplot(x='housing_median_age',data=df)
plt.subplot(3,2,6)
sns.boxplot(x='median_income',data=df)
plt.show()

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR =Q3-Q1
IQR

In [ ]:
df = df[~((df < (Q1-1.5 * IQR)) |(df > (Q3 + 1.5 * IQR))).any(axis=1)]

## Label Encoder Dönüşümü
* Bir özniteliğe ait kategorik sınıflar 0,1,2,3... gibi sayısal değerler ile ifade edilir.
* Bu dönüşümü `ocean_proximity` değikenine kategorik değişken olduğu için uygulayacağız.

In [ ]:
lbe = LabelEncoder().fit_transform(df["ocean_proximity"])
lbe

In [ ]:
df["new_ocean_proximity"] = LabelEncoder().fit_transform(df["ocean_proximity"])
df.head()

In [ ]:
df.drop(columns='ocean_proximity',inplace=True)
df.drop(columns='longitude',inplace=True)
df.drop(columns='latitude',inplace=True)

## Standardizasyon
* Standardizasyon, değerlerin ortalama etrafında bir birim standart sapma ile ortalandığı başka bir ölçeklendirme tekniğidir.
* Değişknelerin ölçeklerinin birbirinden çok farklı olması durumunda bu değişkenlerin analizlere ve modele eşit katkıda bulunamazlar.
* Bu yüzden değişkenlerin standartlaştırılması doğru analiz için önemli bir yere sahiptpir. 
* Örneğin 0–100 aralığında ve 0–1 aralığında yer alan iki değişken üzerinde standardizasyon gerçekleştirilmezse 0–100 aralığındaki değişkenin modeldeki ağırlığı daha fazla olacaktır. 
* Bu sorunu ortadan kaldırmak için verileri kıyaslanabilir ölçekelere dönüştremek gerekir. 
* Veri standardizasyonu ile bu sorunu ortadan kaldırabiliriz.

In [ ]:
std_df=preprocessing.scale(df)
std_df

In [ ]:
std_df=pd.DataFrame(std_df,columns=['housing_median_age',
                                    'total_rooms',
                                   'total_bedrooms','population',
                                   'households',
                                   'median_income',
                                   'median_house_value',
                                   'new_ocean_proximity'])

In [ ]:
std_df.describe().T

In [ ]:
x=std_df.drop(["median_house_value"],axis=1)#bağımsız değişkenler
y=std_df["median_house_value"]#bağımlı değişken

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.40,random_state=42)

## Makine Öğrenmesi
* Veri ön işleme ve veri analizi aşamalarından sonra artık makine öğrenmesi algoritmaları kullanarak modeller kurup tahminler yapabiliriz.
* Bu çalışmada makine öğrenmsi algoritmalarından Rigde Regresyon,Lasso Regresyon,ElasticNet,Xgboost ve Lightgbm modelleri kullanılmıştır.
* Herbir modelin parametrelerinin GridSearchCV yöntemi ile en iyi sonuç veren değerlerin seçilmesine çalışılmıştır. 
* Bu değerler belirlendikten sonra model kurulup tahminler gerçekleştirilmiştir.

## Ridge Modeli

In [ ]:
ridge_model = Ridge().fit(x_train, y_train)

In [ ]:
ridge_alpha= 10**np.linspace(10,-2,100)*0.5

In [ ]:
ridgecv = RidgeCV(alphas =ridge_alpha, scoring = "neg_mean_squared_error", cv = 10, normalize = True).fit(x_train,y_train)

In [ ]:
ridgecv.alpha_

In [ ]:
ridge_tuned = Ridge(alpha = ridgecv.alpha_).fit(x_train,y_train)

In [ ]:
r_y_pred = ridge_tuned.predict(x_test)
r_mse=mean_squared_error(y_test,r_y_pred)
print("Ridge Regresyon Modeli Hata Kareler Ortalaması",r_mse)

## Lasso Regresyon Modeli

In [ ]:
lasso_model = Lasso().fit(x_train,y_train)

In [ ]:
lasso_alphas = 10**np.linspace(20,-2,100)*0.5

In [ ]:
lasso_cv_model = LassoCV(alphas =lasso_alphas, cv = 10, max_iter = 100000).fit(x_train, y_train)

In [ ]:
lasso_cv_model.alpha_

In [ ]:
lasso_tuned = Lasso(alpha =lasso_cv_model.alpha_).fit(x_train, y_train)

In [ ]:
l_y_pred = lasso_tuned.predict(x_test)
l_mse=mean_squared_error(y_test,l_y_pred)
print("Lasso Regresyon Modeli Hata Kareler Ortalaması",l_mse)

## ElasticNet Regresyon Modeli

In [ ]:
enet_model = ElasticNet().fit(x_train,y_train)

In [ ]:
enet_alphas = 10**np.linspace(10,-2,100)*0.5

In [ ]:
enet_cv_model = ElasticNetCV(alphas =enet_alphas, cv = 10).fit(x_train, y_train)

In [ ]:
enet_cv_model.alpha_

In [ ]:
enet_tuned = ElasticNet(alpha = enet_cv_model.alpha_).fit(x_train, y_train)

In [ ]:
e_y_pred = enet_tuned.predict(x_test)
e_mse=mean_squared_error(y_test,e_y_pred)
print("ElasticNet Regresyon Modeli Hata Kareler Ortalaması",e_mse)

## Catboost

In [ ]:
catb_model = CatBoostRegressor().fit(x_train, y_train)

In [ ]:
catb_params = {"iterations": [200,500,1000],
              "learning_rate": [0.01,0.1],
              "depth": [3,6,8]}

In [ ]:
catb_cv_model = GridSearchCV(catb_model, 
                           catb_params, 
                           cv = 5, 
                           n_jobs = -1, 
                           verbose = 2).fit(x_train, y_train)

In [ ]:
catb_cv_model.best_params_

In [ ]:
catb_tuned = CatBoostRegressor(depth = 8, iterations =200, learning_rate = 0.1).fit(x_train, y_train)

In [ ]:
c_y_pred = catb_tuned.predict(x_test)
c_mse=mean_squared_error(y_test,c_y_pred)
print("CatBoost Regresyon Modeli Hata Kareler Ortalaması",c_mse)

# KNN-NEİGHBORS

In [ ]:
knn_model=KNeighborsRegressor().fit(x_train,y_train)

In [ ]:
knn_params={"n_neighbors":np.arange(1,100,1)}

In [ ]:
knn_cv_model=GridSearchCV(knn_model,knn_params,cv=10).fit(x_train,y_train)

In [ ]:
knn_cv_model.best_params_

In [ ]:
knn_tuned=KNeighborsRegressor(n_neighbors=knn_cv_model.best_params_["n_neighbors"]).fit(x_train,y_train)

In [ ]:
knn_y_pred=knn_tuned.predict(x_test)
knn_mse=mean_squared_error(y_test, knn_y_pred)
print("KNeighbors Regresyon Modeli Hata Kareler Ortalaması",knn_mse)

## LightGbm Modeli

In [ ]:
lgbm = LGBMRegressor()
lgbm_model=lgbm.fit(x_train, y_train)

In [ ]:
lgbm_params = {"learning_rate": [0.01, 0.1, 0.5, 1],
              "n_estimators": [20,40,100,200,500,1000],
              "max_depth": [1,2,3,4,5,6,7,8,9,10]}

In [ ]:
lgbm_cv_model = GridSearchCV(lgbm, 
                             lgbm_params, 
                             cv = 10, 
                             n_jobs = -1, 
                             verbose =2).fit(x_train, y_train)

In [ ]:
print(lgbm_cv_model.best_params_)

In [ ]:
lgbm_tuned = LGBMRegressor(learning_rate = 0.01, 
                          max_depth = 10, 
                          n_estimators = 1000).fit(x_train, y_train)

In [ ]:
lgbm_y_pred = lgbm_tuned.predict(x_test)
lgbm_mse=mean_squared_error(y_test, lgbm_y_pred)
print("LightGbm Modeli Hata Kareler Ortalaması",lgbm_mse)

## XGBOOST Modeli

In [ ]:
xgb = XGBRegressor().fit(x_train,y_train)

In [ ]:
xgb_params = {"learning_rate": [0.1,0.01,0.5],
             "max_depth": [2,3,4,5,8],
             "n_estimators": [100,200,500,1000],
             "colsample_bytree": [0.4,0.7,1]}

In [ ]:
xgb_cv=GridSearchCV(xgb,xgb_params,cv=5,n_jobs=-1,verbose=2).fit(x_train,y_train)

In [ ]:
xgb_cv.best_params_

In [ ]:
xgb_tuned=XGBRegressor(colsample_bytree=1,
                       learning_rate=0.1,
                       max_depth=5,
                       n_estimators=200).fit(x_train,y_train)
xgb_y_pred=xgb_tuned.predict(x_test)

In [ ]:
xgb_mse=mean_squared_error(y_test,xgb_y_pred)
print("XGBOOST Modeli Hata Kareler Ortalaması:",xgb_mse)

## Sonuç
* Ridge Regresyon Modeli Hata Kareler Ortalaması: 0.4719641495788811
* Lasso Regresyon Modeli Hata Kareler Ortalaması: 0.4734783036550058
* ElasticNet Regresyon Modeli Hata Kareler Ortalaması: 0.47277822616262133
* KNeighbors Regresyon Modeli Hata Kareler Ortalaması: 0.36970662593093034
* XGBOOST Modeli Hata Kareler Ortalaması: 0.34783882157774626
* LightGbm Modeli Hata Kareler Ortalaması 0.340259617148958
* CatBoost Regresyon Modeli Hata Kareler Ortalaması: 0.3353974207160476